## Prepare Dataset

In [ ]:
!mkdir /content/datasets

In [ ]:
!cp /content/drive/MyDrive/datasets/reid/DukeMTMC-reID.zip -d /content/datasets
!cp /content/drive/MyDrive/datasets/reid/Market-1501-v15.09.15.zip -d /content/datasets
!cp "/content/drive/MyDrive/datasets/reid/REID CANIFA/20220731_images_crop.zip" -d /content/datasets

In [ ]:
!mkdir /content/datasets/market1501
!unzip -qq /content/datasets/Market-1501-v15.09.15.zip -d /content/datasets/market1501

In [ ]:
!mkdir /content/datasets/dukemtmc
!unzip -qq /content/datasets/DukeMTMC-reID.zip -d /content/datasets/dukemtmc

In [ ]:
!mkdir -p /content/datasets/canifa
!unzip -qq /content/datasets/20220731_images_crop.zip -d /content/datasets/canifa

In [ ]:
!rm -rf "/content/datasets/canifa/Camera13_images_crop" \
        "/content/datasets/canifa/Camera5_images_crop" \
        "/content/datasets/canifa/Camera7_images_crop" \
        "/content/datasets/canifa/Camera8_images_crop"
        

## Install dependences

In [ ]:
!pip install yacs faiss-cpu

## Training

In [ ]:
!cd /content/drive/MyDrive/colab/reid/SECRET/secret/metrics/rank_cylib \
  && python3 setup.py build_ext --inplace \
  && cd /content/

### Phase 1: Create pretrain

In [ ]:
!python /content/drive/MyDrive/colab/reid/SECRET/main.py \
  --config-file /content/drive/MyDrive/colab/reid/SECRET/configs/pretrain.yml \
  DATASETS.DIR "/content/datasets" \
  DATASETS.SOURCE "market1501" \
  DATASETS.TARGET "dukemtmc" \
  OUTPUT_DIR "/content/drive/MyDrive/colab/reid/SECRET/log/market2duke/pretrain" \
  GPU_Device '[0]' \
  MODE 'pretrain' \
  DATALOADER.ITERS 200 \
  MODEL.ARCH "resnet50"

### Phase 2: Clustering

In [ ]:
!python /content/drive/MyDrive/colab/reid/SECRET/main.py \
  --config-file /content/drive/MyDrive/colab/reid/SECRET/configs/mutualrefine.yml \
  DATASETS.DIR "/content/datasets" \
  DATASETS.SOURCE "dukemtmc" \
  DATASETS.TARGET "canifa" \
  CHECKPOING.PRETRAIN_PATH "/content/drive/MyDrive/colab/reid/SECRET/pretrain/checkpoint_new_market78.pth.tar" \
  OUTPUT_DIR "/content/drive/MyDrive/colab/reid/SECRET/log/duke2canifa/mutualrefine" \
  GPU_Device '[0]' \
  MODE 'mutualrefine' \
  MODEL.ARCH "resnet50" OPTIM.EPOCHS 50 \
  OPTIM.LR 0.0000875 DATALOADER.BATCH_SIZE 16 DATALOADER.ITERS 5000